## CREATE

In [79]:
from sqlalchemy import Column, Integer, Boolean, VARCHAR
from sqlalchemy.sql.expression import false
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base


user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"

Base = declarative_base()
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)


class TaskTable(Base):
    __tablename__ = "tasks"

    id_number = Column("id", Integer, primary_key=True)  # auto-increment included
    description = Column("description", VARCHAR(30), nullable=False)
    priority = Column("priority", Integer)
    is_complete = Column("is_complete", Boolean, nullable=False, server_default=false())
    
    def __repr__(self):
        return f"Task(id={self.id_number})"


    
class UserTable(Base):
    __tablename__ = "users"

    id_number = Column("id", Integer, primary_key=True)
    username = Column("username", VARCHAR(20), nullable=False)
    password = Column("password", VARCHAR(30), nullable=False)
    is_admin = Column("is_admin", Boolean, nullable=False, server_default=false())
    
    def __repr__(self):
        return f"User(id={self.id_number})"

In [80]:
Base.metadata.create_all(bind=engine)

## INSERT

In [81]:
description = "Task description123"
priority = 2
is_complete = True

task = TaskTable(description=description, priority=priority, is_complete=is_complete)

In [82]:
session = Session()

In [83]:
session.add(task)

In [84]:
session.commit()

## SELECT

### All data

In [19]:
tasks_data = session.query(TaskTable).all()
tasks_data

[Task(id=1), Task(id=2), Task(id=3)]

### One row

In [23]:
session.query(TaskTable).first()

Task(id=1)

### Select columns

In [24]:
session.query(TaskTable.description, TaskTable.priority).filter_by(is_complete=False).all()

[('Task description1', 3), ('Task description12', 31)]

### Filter (WHERE)

In [25]:
from sqlalchemy import between

In [26]:
tasks_data = session.query(TaskTable).filter_by(is_complete=False).all()
tasks_data

[Task(id=1), Task(id=2)]

In [27]:
session.query(TaskTable).filter(between(TaskTable.priority, 3, 40)).all()

[Task(id=1), Task(id=2)]

### Sort

In [28]:
from sqlalchemy import asc, desc

In [30]:
session.query(TaskTable).order_by(asc(TaskTable.priority)).all()

[Task(id=3), Task(id=1), Task(id=2)]

In [31]:
session.query(TaskTable).order_by(desc(TaskTable.priority)).all()

[Task(id=2), Task(id=1), Task(id=3)]

---

In [32]:
from sqlalchemy import func

In [37]:
session.query(TaskTable).order_by(desc(func.length(TaskTable.description))).all()

[Task(id=3), Task(id=2), Task(id=1)]

In [38]:
session.query(TaskTable).order_by(asc(func.length(TaskTable.description))).all()

[Task(id=1), Task(id=2), Task(id=3)]

### Groupby

In [55]:
session.query(TaskTable.is_complete, func.count(TaskTable.id_number)).group_by(TaskTable.is_complete).all()

[(False, 2), (True, 1)]

In [56]:
session.query(TaskTable.is_complete, func.avg(TaskTable.priority)).group_by(TaskTable.is_complete).all()

[(False, Decimal('17.0000000000000000')),
 (True, Decimal('2.0000000000000000'))]

## UPDATE

In [61]:
record_to_update = session.query(TaskTable).filter(TaskTable.id_number == 2).one()  # one() instead of first() if only one result expected
record_to_update

Task(id=2)

In [62]:
record_to_update.is_complete = True

In [63]:
session.commit()

### DELETE

In [64]:
record_to_delete = session.query(TaskTable).filter(TaskTable.id_number == 2).one()
record_to_delete

Task(id=2)

In [66]:
session.delete(record_to_delete)

In [67]:
session.commit()

### DROP

In [72]:
from sqlalchemy import Table, MetaData

In [76]:
metadata = MetaData()
table_to_drop = Table('tasks', metadata, autoload_with=engine)

In [78]:
table_to_drop.drop(engine)